In [28]:
import pandas as pd  
import numpy as np
import re
import random
from unidecode import unidecode 

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 

from time import sleep  

In [ ]:
archivo_csv = "cortos_drama_2015_2019.csv"  # Ruta del archivo CSV
nombre_columna_titulos = "1"  # Nombre de la columna de los títulos de las películas
nombre_columna_anios = "2"  # Nombre de la columna de los años de las películas

# Cargar el archivo CSV
peliculas_df = pd.read_csv(archivo_csv)

# Iniciar el navegador Chrome
driver = webdriver.Chrome()

# Abrir la página de IMDb
driver.get("https://www.imdb.com/")
driver.maximize_window()
sleep(3)

try:
    # Aceptar las cookies si es necesario
    driver.find_element("css selector" , "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-hLBbgP.sc-ftTHYK.dcvrLS.dufgkr.ecppKW").click()
except:
    pass

sleep(random.randint(1, 3))

print("--- Empieza la extracción de datos ---")
print(" ")

lista_detalles_peliculas = []
lista_actores = []

for indice, pelicula in peliculas_df.iterrows():
    nombre_peli = pelicula["1"].lower()  # Obtener el nombre de la película
    anio_peli = int(pelicula["2"])  # Obtener el año de la película
    string_buscar = f"{nombre_peli}, {anio_peli}"  # Definir la frase de búsqueda
    encontrada = False
    
    # Establecemos variables de inicio
    puntuacion_encontrada = False
    reparto_encontrado = False
    duracion_encontrada = False

    # Realiza la búsqueda en IMDb
    try:  
        driver.find_element("xpath", '//*[@id="suggestion-search"]').clear()
        driver.find_element("xpath", '//*[@id="suggestion-search"]').send_keys(string_buscar, Keys.ENTER)
    except:
        print("No se encuentra la barra de búsqueda")
        break
    
    sleep(random.randint(1, 2))

    # Itera sobre los resultados de la búsqueda
    for i in range(1, 6):
        try:
            # Obtiene el título y el año del resultado actual
            datos_resultado = driver.find_element("xpath", f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[2]').text.lower()
            titulo_resultado = datos_resultado.split('\n')[0].strip()
            anio_resultado = int(re.findall('\d{4}' , datos_resultado.split('\n')[1].strip())[0])
                            
        except:
            print("Error al obtener datos del resultado")
            break
        
        if unidecode(nombre_peli) == unidecode(titulo_resultado) and anio_peli == anio_resultado:
            # Hace click en el resultado donde se encuentran las coincidencias de título y año
            driver.find_element("xpath" , f'//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[{i}]/div[1]/div').click()
            
            sleep(random.randint(2, 3))
            
            info = driver.find_element('xpath' , f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section').text 
            info_separada = info.split('\n')
            
            # Itera por los elementos de la cabecera
            for indice , elemento in enumerate(info_separada):
                
                if unidecode(elemento.lower()) == 'direccion':
                    datos_direccion = info_separada[indice+1]
                    
                    # Extrae todas las coincidencias de nombre y apellidos de la string
                    totales_d = re.findall('[A-Z][a-z]*', datos_direccion)
                    
                    # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                    if len(totales_d) > 2 and len(totales_d) % 2 == 0:
                        datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d), 2)]  
                        
                    elif len(totales_d) % 2 != 0:
                        # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo
                        datos_direccion = [f"{totales_d[i]} {totales_d[i+1]}" for i in range(0, len(totales_d) - 2, 2)]
                        
                        # Agrega el último elemento al penúltimo
                        datos_direccion[-1] += f" {totales_d[-1]}"                       
                
                if unidecode(elemento.lower()) == 'guion':
                    datos_guion = info_separada[indice+1]
                    
                    # Extrae todas las coincidencias de nombre y apellidos de la string
                    totales_g = re.findall('[A-Z][a-z]*', datos_guion)
                    
                    # Si hay más de 1 nombre y 1 apellido, entonces guarda en una lista la combinación de todos ellos
                    if len(totales_g) > 2 and len(totales_g) % 2 == 0:
                        datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g), 2)]
                    
                    elif len(totales_g) % 2 != 0:
                        # Si la lista de nombres y apellidos es impar, combina todos los elementos excepto el último y añade el último al penúltimo
                        datos_guion = [f"{totales_g[i]} {totales_g[i+1]}" for i in range(0, len(totales_g) - 2, 2)]
                        
                        # Agrega el último elemento al penúltimo
                        datos_guion[-1] += f" {totales_g[-1]}"
                    
                if 'argumento' in elemento.lower():
                    argumento = info_separada[indice+1]
                    
                if 'puntuacion en imdb' in elemento.lower():
                    puntuacion = info_separada[indice+1]
                    puntuacion_encontrada = True
                
                if 'reparto' in elemento.lower():
                    reparto_encontrado = True
                    
                if any(char.isdigit() for char in elemento) and 'min' in elemento.lower():
                    horas = info_separada[indice]   
                    duracion_encontrada = True    
                                
                    if len(horas.split()) > 1:
                        duracion_minutos = int(horas.split()[0].replace('h' , ''))*60 + int(horas.split()[1].replace('min', ''))
                    
                    else:
                        duracion_minutos = int(horas.replace('min', '')) 
            
            # Si no había datos de puntuación, asigna '-' por defecto            
            if not puntuacion_encontrada:
                puntuacion = '-'
            
            # Si no había datos de duración, asigna '-' por defecto     
            if not duracion_encontrada:
                duracion_minutos = '-' 
            
            # Si encontró datos del reparto en el resumen de la película, intenta extraerlos de la página
            if reparto_encontrado:
                
                try:
                    # Extrae los datos del reparto principal
                    datos_reparto = driver.find_element("xpath" , f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]').text
                    
                    # Separa los datos obtenidos y se queda sólo con los nombres de los actores
                    datos_reparto = datos_reparto.split('\n')[::2] 
                    
                    # Si la longitud de la lista de actores obtenida es superior a 10, la limitamos 
                    if len(datos_reparto) > 10:
                        datos_reparto = datos_reparto[:10]
                        
                except:
                    datos_reparto = np.nan
                    
            # Cambia la variable a positiva, ya que ha encontrado la película en IMDB
            encontrada = True
            break
    
    # Si encontró la película en IMDB:
    if encontrada:
        # Crea una tupla con todos los detalles de la película
        tupla_peli = (puntuacion , datos_direccion , datos_guion , argumento , duracion_minutos , nombre_peli)
        print(tupla_peli)
        
        # Añade la tupla al listado de películas
        lista_detalles_peliculas.append(tupla_peli)
        
        # Añade el listado de actores a la lista
        lista_actores.extend(datos_reparto)
    
    # Si no la encontró, crea una tupla con resultados vacíos y la peli    
    else:
        tupla_peli = (np.nan , np.nan , np.nan , np.nan , np.nan , nombre_peli)
        
        # Añade la tupla al listado de películas
        lista_detalles_peliculas.append(tupla_peli)
    
    sleep(random.randint(1, 2))           
   